In [8]:
#Load the libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from pandas_profiling import ProfileReport
import plotly as pot
import plotly.express as px
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split


In [22]:
data=pd.read_csv("/Users/himanshusahrawat/ALP-1/datasets/loan/loan_20col.csv")
data.head()

,funded_amnt,term,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,open_acc,pub_rec,revol_util,total_acc,application_type,acc_now_delinq,tot_cur_bal
0,5000,36,162.87,B,B2,10,RENT,24000,Verified,0,credit_card,27.65,3,0,83.70,9,INDIVIDUAL,0,139365
1,2500,60,59.83,C,C4,0,RENT,30000,Source Verified,1,car,1.00,3,0,9.40,4,INDIVIDUAL,0,139365
2,2400,36,84.33,C,C5,10,RENT,12252,Not Verified,0,small_business,8.72,2,0,98.50,10,INDIVIDUAL,0,139365
3,10000,36,339.31,C,C1,10,RENT,49200,Source Verified,0,other,20.00,10,0,21.00,37,INDIVIDUAL,0,139365
4,3000,60,67.79,B,B5,1,RENT,80000,Source Verified,0,other,17.94,15,0,53.90,38,INDIVIDUAL,0,139365


In [24]:
a=data["loan_status"].value_counts()
print((a[0])/a.sum())

0.9233145875549281


In [25]:
a

0    811058
1     67362
Name: loan_status, dtype: int64

Dataset is highly imbalanced.

In [26]:
#taking 50k data with startified sampling

from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, train_size=50000, random_state=42)
for train_index, test_index in split.split(data, data['loan_status']):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]
    
    
#checking the 50k dataset distribution

a=strat_train_set["loan_status"].value_counts()
print((a[1])/a.sum())

0.07668


In [27]:
### separating dataset into dependent and independent features, testing algos on 50k data to get an idea of model
### performance. 
y = strat_train_set["loan_status"]
x = strat_train_set.drop(columns=["loan_status"])

print("x shape :", x.shape)

print("y shape :", y.shape)

x shape : (50000, 18)
y shape : (50000,)


In [28]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state =123)

In [29]:
print("X_train shape :", X_train.shape)

print("X_test shape :", X_test.shape)

print("y_train shape :", y_train.shape)

print("y_test shape :", y_test.shape)

X_train shape : (35000, 18)
X_test shape : (15000, 18)
y_train shape : (35000,)
y_test shape : (15000,)


In [33]:
a=y_test.value_counts()
a

0    13846
1     1154
Name: loan_status, dtype: int64

In [10]:
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric = None)
### fitting data in LazyClassifier
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
### lets check model performance
print(models)

100%|███████████████████████████████████████████| 29/29 [06:04<00:00, 12.58s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
NearestCentroid                    0.63               0.60     0.60      0.72   
Perceptron                         0.86               0.55     0.55      0.86   
LabelSpreading                     0.86               0.52     0.52      0.86   
LabelPropagation                   0.86               0.52     0.52      0.86   
DecisionTreeClassifier             0.85               0.52     0.52      0.86   
ExtraTreeClassifier                0.86               0.51     0.51      0.86   
GaussianNB                         0.10               0.51     0.51      0.06   
KNeighborsClassifier               0.92               0.50     0.50      0.89   
XGBClassifier                      0.92               0.50     0.50      0.89   
LinearDiscriminantAnalysis         0.92               0.50     0.50      0.89   
LGBMClassifier              

We have to check if the model is not getting overfiting and we will try to improve the accuracy. As you can see, only about 7.6% of the observations were balanced. Therefore, if we were to always predict 0, we’d achieve an accuracy of 92%. 

## Downsampling

#### Structure
.
.

.

.

.

.

.

.....

In [103]:
data=pd.read_csv("/Users/himanshusahrawat/ALP-1/datasets/loan/loan_20_outliers.csv")
data.head()

,funded_amnt,term,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,open_acc,pub_rec,revol_util,total_acc,application_type,acc_now_delinq,tot_cur_bal
0,5000,36,162.87,B,B2,10,RENT,24000,Verified,0,credit_card,27.65,3,0,83.70,9,INDIVIDUAL,0,139365
1,2500,60,59.83,C,C4,0,RENT,30000,Source Verified,1,car,1.00,3,0,9.40,4,INDIVIDUAL,0,139365
2,2400,36,84.33,C,C5,10,RENT,12252,Not Verified,0,small_business,8.72,2,0,98.50,10,INDIVIDUAL,0,139365
3,10000,36,339.31,C,C1,10,RENT,49200,Source Verified,0,other,20.00,10,0,21.00,37,INDIVIDUAL,0,139365
4,3000,60,67.79,B,B5,1,RENT,80000,Source Verified,0,other,17.94,15,0,53.90,38,INDIVIDUAL,0,139365


In [104]:
# break the whole data into 95%(training for downsampling)-5%(test).

train, test = train_test_split(data, test_size=.05, random_state =123)

print("train shape :", train.shape)

print("test shape :", test.shape)

train shape : (808533, 19)
test shape : (42555, 19)


In [105]:
#check the balancing of train dataset

a=train["loan_status"].value_counts()
a

0    746258
1     62275
Name: loan_status, dtype: int64

In [106]:
from sklearn.utils import resample
# downsample the majority class 

df_majority = train[train.loan_status==0]
df_minority = train[train.loan_status==1]
 

df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=160000,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
train_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
train_downsampled.loan_status.value_counts()

0    160000
1     62275
Name: loan_status, dtype: int64

In [107]:
# taking 50k data only for traning

#taking 50k data with startified sampling

train_35k, test = train_test_split(train_downsampled, train_size=35000, random_state =32)

# 20k test data
split = StratifiedShuffleSplit(n_splits=1, test_size=15000, random_state=42)
for train_index, test_index in split.split(test, test['loan_status']):
    strat_test_15k = data.loc[test_index]

In [108]:
train_35k["loan_status"].value_counts()

0    25307
1     9693
Name: loan_status, dtype: int64

In [109]:
# spliting the Y variable

X_train= train_35k.drop(columns=["loan_status"])

X_test= strat_test_15k.drop(columns=["loan_status"])

y_train= train_35k["loan_status"]

y_test= strat_test_15k["loan_status"]

print("X_train shape :", X_train.shape)

print("X_test shape :", X_test.shape)

print("y_train shape :", y_train.shape)

print("y_test shape :", y_test.shape)

X_train shape : (35000, 18)
X_test shape : (15000, 18)
y_train shape : (35000,)
y_test shape : (15000,)


In [110]:
a=y_train.value_counts()
a

0    25307
1     9693
Name: loan_status, dtype: int64

Data is balanced now, lets test the raw models.

In [83]:
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric = None)
### fitting data in LazyClassifier
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
### lets check model performance
print(models)

100%|███████████████████████████████████████████| 29/29 [13:36<00:00, 28.16s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
RandomForestClassifier             0.78               0.65     0.65      0.80   
ExtraTreesClassifier               0.78               0.64     0.64      0.80   
BaggingClassifier                  0.76               0.63     0.63      0.78   
XGBClassifier                      0.72               0.62     0.62      0.76   
NearestCentroid                    0.65               0.62     0.62      0.71   
LinearSVC                          0.73               0.61     0.61      0.76   
DecisionTreeClassifier             0.64               0.61     0.61      0.69   
LGBMClassifier                     0.75               0.60     0.60      0.78   
LabelSpreading                     0.65               0.60     0.60      0.70   
LabelPropagation                   0.65               0.60     0.60      0.70   
SGDClassifier               

In [102]:
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric = None)
### fitting data in LazyClassifier
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
### lets check model performance
print(models)

100%|███████████████████████████████████████████| 29/29 [17:25<00:00, 36.05s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.82               0.62     0.62      0.82   
RandomForestClassifier             0.83               0.62     0.62      0.83   
BaggingClassifier                  0.80               0.61     0.61      0.81   
NearestCentroid                    0.65               0.60     0.60      0.70   
PassiveAggressiveClassifier        0.61               0.60     0.60      0.67   
DecisionTreeClassifier             0.67               0.60     0.60      0.72   
SGDClassifier                      0.72               0.59     0.59      0.75   
GaussianNB                         0.70               0.59     0.59      0.74   
LabelSpreading                     0.68               0.59     0.59      0.73   
LabelPropagation                   0.68               0.59     0.59      0.73   
ExtraTreeClassifier         

In [111]:
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric = None)
### fitting data in LazyClassifier
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
### lets check model performance
print(models)

100%|███████████████████████████████████████████| 29/29 [16:53<00:00, 34.94s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
RandomForestClassifier             0.83               0.61     0.61      0.82   
ExtraTreesClassifier               0.82               0.60     0.60      0.82   
GaussianNB                         0.70               0.60     0.60      0.74   
NearestCentroid                    0.65               0.60     0.60      0.70   
BaggingClassifier                  0.79               0.60     0.60      0.80   
DecisionTreeClassifier             0.67               0.59     0.59      0.72   
ExtraTreeClassifier                0.67               0.58     0.58      0.72   
LabelSpreading                     0.68               0.58     0.58      0.72   
LabelPropagation                   0.68               0.58     0.58      0.72   
XGBClassifier                      0.78               0.58     0.58      0.79   
PassiveAggressiveClassifier 

After balancing the target variable, accuracy has increased. Now, we will try to fit the data into ensemblers after feature encoding.